In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
from larndsim import detsim

import importlib
from larndsim import consts, quenching, drifting, TPC, pixel_response, diffused_charge
importlib.reload(detsim)
importlib.reload(drifting)
importlib.reload(quenching)
importlib.reload(TPC)
importlib.reload(pixel_response)
importlib.reload(diffused_charge)

import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

import pickle
import numpy as np
import numba as nb

import math
import pandas as pd

In [3]:
tracks = pickle.load(open('FakeTrackDataSet.p', 'rb'))
tracks['t'] = 0.
tracks['t_start'] = 0.
tracks['t_end'] = 0.

tracks['dx'] = np.sqrt(pow(tracks['x_end']-tracks['x_start'], 2) +
                    pow(tracks['y_end']-tracks['y_start'], 2) +
                    pow(tracks['z_end']-tracks['z_start'], 2))
tracks['x'] = (tracks['x_end']+tracks['x_start'])/2
tracks['y'] = (tracks['y_end']+tracks['y_start'])/2
tracks['z'] = (tracks['z_end']+tracks['z_start'])/2
tracks['dE'] = np.abs(tracks['dE'])*1e3
tracks['dEdx'] = tracks['dE']/tracks['dx']
tracks['NElectrons'] = 0.
tracks['longDiff'] = 0.
tracks['tranDiff'] = 0.

In [4]:
tracks_np = tracks.to_numpy()
cols = nb.typed.Dict()
for i, c, in enumerate(tracks.columns):
    cols[c] = i

In [5]:
quenching.Quench(tracks_np, cols)

In [6]:
drifting.Drift(tracks_np, cols)

In [7]:
selectedTracks = tracks_np[(tracks_np[:,cols['trackID']] > 6) & (tracks_np[:,cols['trackID']] < 9)]

In [8]:
PixelSignal = pd.DataFrame(columns = ['track_id','pixel_id','t_range','z_range','current'])

for track in selectedTracks:
    
    pixelIDs = pixel_response.getPixels(track, cols)
    zIntervals = pixel_response.getZInterval(track, cols, pixelIDs)
    weights = diffused_charge.TrackCharge(track, cols, zIntervals)
    pixel_signal = pixel_response.getPixelSignal(track, cols, pixelIDs, zIntervals, weights)
    
    t_start = (track[cols['t_start']]-20) // TPC.t_sampling * TPC.t_sampling
    t_end = (track[cols['t_end']]+20) // TPC.t_sampling * TPC.t_sampling
    t_range = (t_start, t_end)
    
    pixel_df = pd.DataFrame(columns = ['track_id','pixel_id','t_range','z_range','current'])
    pixel_df['pixel_id'] = list(pixelIDs)
    pixel_df['z_range'] = list(zIntervals)
    pixel_df['t_range'] = [t_range] * pixel_df.shape[0]
    pixel_df['track_id'] = [track[cols['trackID']]] * pixel_df.shape[0]
    pixel_df['current'] = pixel_signal
    
    PixelSignal = pd.concat([PixelSignal, pixel_df], ignore_index = False, sort = False)

PixelSignal = PixelSignal.reset_index(drop = True)

In [9]:
PixelSignal

,track_id,pixel_id,t_range,z_range,current
0,7.0,"[97, 700]","(1306.5, 1482.5)","[54.07216604728626, 54.643392138921875]","[198.08571111575137, 222.4986776161096, 249.95..."
1,7.0,"[97, 701]","(1306.5, 1482.5)","[54.07216604728626, 54.80490693389009]","[166.00642563954864, 188.0243396033396, 212.99..."
2,7.0,"[98, 700]","(1306.5, 1482.5)","[54.07216604728626, 54.72380503676886]","[112.78389252653025, 128.83401289359006, 147.1..."
3,7.0,"[97, 699]","(1306.5, 1482.5)","[54.07216604728626, 54.44771812479305]","[79.97591420999754, 91.30108479897659, 104.247..."
4,7.0,"[96, 700]","(1306.5, 1482.5)","[54.07216604728626, 54.49335181906321]","[92.06010654393671, 104.91700770794108, 119.58..."
...,...,...,...,...,...
739,8.0,"[204, 809]","(1176.5, 1351.5)","[54.50038046751879, 54.80836654123566]","[2.4447982149688277e-13, 2.749883704719429e-13..."
740,8.0,"[205, 808]","(1176.5, 1351.5)","[54.544202892994065, 54.80836654123566]","[1.4780997215344684e-14, 1.6760726265631515e-1..."
741,8.0,"[205, 809]","(1176.5, 1351.5)","[54.63746334954198, 54.80836654123566]","[1.9266445843207218e-15, 2.196122140163857e-15..."
742,8.0,"[205, 807]","(1176.5, 1351.5)","[54.52831040027958, 54.80836654123566]","[2.9691467545592667e-16, 3.4212980363069794e-1..."


In [10]:
%matplotlib widget
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

PixelSignal['sum_current'] = PixelSignal['current'].apply(np.sum)
currents = np.array(PixelSignal['sum_current'])
norm = mpl.colors.LogNorm(vmin=min(currents[currents>0]), vmax=max(currents))
cmap = cm.viridis
m = cm.ScalarMappable(norm=norm, cmap=cmap)

for i, each in PixelSignal.iterrows():
    rect = plt.Rectangle((each['pixel_id'][0]*TPC.x_pixel_size+consts.tpcBorders[0][0],
                          each['pixel_id'][1]*TPC.y_pixel_size+consts.tpcBorders[1][0]),
                         TPC.x_pixel_size,TPC.y_pixel_size,
                         linewidth=0,
                         edgecolor='grey', 
                         facecolor=m.to_rgba(each['sum_current']), label='Pixel')
    ax.add_patch(rect)
    art3d.pathpatch_2d_to_3d(rect, z=150, zdir="z")

for t in selectedTracks:
    ax.plot((t[cols['x_start']], t[cols['x_end']]), 
            (t[cols['y_start']], t[cols['y_end']]),
            (t[cols['z_start']], t[cols['z_end']]),
            c='r')
    ax.scatter((t[cols['x_start']], t[cols['x_end']]), 
               (t[cols['y_start']], t[cols['y_end']]),
               (t[cols['z_start']], t[cols['z_end']]),
               c='r')

ax.set_xlabel("$x$ coordinate [cm]")
ax.set_ylabel("$y$ coordinate [cm]")
ax.set_zlabel("$z$ coordinate [cm]")
ax.set_zlim3d(150,50)
ax.set_xlim3d(-80,-130)
ax.set_ylim3d(55,100)

_ = plt.colorbar(m,fraction=0.046, pad=0.04, label='Current integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/zoya/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:509: UserWarning: Warning: converting a masked element to nan.
  xa = np.array([X])


In [11]:
fig,ax = plt.subplots(1,1, constrained_layout=True)

for i, each in PixelSignal.iterrows():
    rect = plt.Rectangle((each['pixel_id'][0]*TPC.x_pixel_size+consts.tpcBorders[0][0],
                          each['pixel_id'][1]*TPC.y_pixel_size+consts.tpcBorders[1][0]),
                         TPC.x_pixel_size,TPC.y_pixel_size,
                         linewidth=0.3,
                         edgecolor='grey', 
                         facecolor=m.to_rgba(each['sum_current']), label='Pixel')
    ax.add_patch(rect)
    
for t in selectedTracks:
    ax.plot((t[cols['x_start']], t[cols['x_end']]), 
            (t[cols['y_start']], t[cols['y_end']]),
            c='r',
            zorder=10)

ax.set_xlabel("$x$ coordinate [cm]")
ax.set_ylabel("$y$ coordinate [cm]")
ax.set_aspect("equal")
_ = plt.colorbar(m,fraction=0.046, pad=0.04, label='Current integral')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/Users/zoya/opt/anaconda3/lib/python3.7/site-packages/matplotlib/colors.py:509: UserWarning: Warning: converting a masked element to nan.
  xa = np.array([X])


In [12]:
fig, ax = plt.subplots(1,1, constrained_layout=True)

pixelID = pixel_response.getPixelFromCoordinates(-111.8,72.3)
current = np.zeros_like(TPC.anode_t)
temp = PixelSignal.loc[[(i == pixelID[0] and j == pixelID[1]) for i,j in PixelSignal['pixel_id']],['t_range','current']]
for i, each in temp.iterrows():
    current[(TPC.anode_t >= each['t_range'][0]) 
            &(TPC.anode_t <= each['t_range'][1])] += each['current']

ax.plot(TPC.anode_t, current, label='Current')
ax.set_xlim(1100,1500) 
ax.set_title("Pixel (%i,%i) at (%.2f, %2.f)" % (pixelID[0], 
                                      pixelID[1], 
                                      (pixelID[0] + 0.5)*TPC.x_pixel_size+consts.tpcBorders[0][0], 
                                      (pixelID[1] + 0.5)*TPC.y_pixel_size+consts.tpcBorders[1][0]))
ax.set_xlabel("Time [$\mathrm{\mu}$s]")
ax.set_ylabel("Current [a.u.]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Current [a.u.]')